# Loyalty Program Analysis Decomposition

This analysis is for a building material retailer called Homeworld. We were tasked to determine if the loyalty program helped boost purchased when it was introduced. We will do some data pre-processing, EDA, and also test whether the loyaltry program increased revenue. 

### Description of the data

The dataset contains data on purchases made at the building-material retailer Home World. All of its customers have membership cards. Moreover, they can become members of the store's loyalty program for $20 per month. The program includes discounts, information on special offers, and gifts. 

`retail_dataset_us.csv` contains:

- `purchaseId`
- `item_ID`
- `purchasedate`
- `Quantity` — the number of items in the purchase
- `CustomerID`
- `ShopID`
- `loyalty_program` — whether the customer is a member of the loyalty program

`product_codes_us.csv` contains:

- `productID`
- `price_per_one`

### Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#reading in the data
products = pd.read_csv('product_codes_us.csv', sep=';')
retail = pd.read_csv('retail_dataset_us.csv')

In [3]:
#looking at the dataset
products.head()

,productID,price_per_one
0,10002,0.85
1,10080,0.85
2,10120,0.21
3,10123C,0.65
4,10124A,0.42


In [4]:
#changing column names
products.columns = ['product_id', 'price_per_one']

In [5]:
products.head()

,product_id,price_per_one
0,10002,0.85
1,10080,0.85
2,10120,0.21
3,10123C,0.65
4,10124A,0.42


In [41]:
products.describe()

,price_per_one
count,3159.000000
mean,2.954495
std,7.213808
min,0.000000
25%,0.650000
50%,1.450000
75%,3.290000
max,175.000000


We can see the average cost for a product is 2.95. The highest cost of a product is 175. The standard deviation is 7.21 which means that there are some values far from the mean (there is some variety of prices here). 

In [6]:
#looking at the data types
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3159 entries, 0 to 3158
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     3159 non-null   object 
 1   price_per_one  3159 non-null   float64
dtypes: float64(1), object(1)
memory usage: 49.5+ KB


In [9]:
#looking for missing values
products.isnull().sum()

product_id       0
price_per_one    0
dtype: int64

In [11]:
#looking at duplicates
products.duplicated().sum()

0

In [20]:
#making sure there are no implicit duplicates
products['product_id'].duplicated().sum()

0

In [22]:
#looking at retail dataset
retail.head()

,purchaseid,item_ID,Quantity,purchasedate,CustomerID,loyalty_program,ShopID
0,538280,21873,11,2016-12-10 12:50:00,18427.0,0,Shop 3
1,538862,22195,0,2016-12-14 14:11:00,22389.0,1,Shop 2
2,538855,21239,7,2016-12-14 13:50:00,22182.0,1,Shop 3
3,543543,22271,0,2017-02-09 15:33:00,23522.0,1,Shop 28
4,543812,79321,0,2017-02-13 14:40:00,23151.0,1,Shop 28


In [28]:
#changing column names
retail.columns =['purchase_id', 'item_id', 'quantity', 'purchase_date', 'customer_id', 'loyalty_program', 'shop_id']

In [29]:
#making sure column names are changed 
retail.head()

,purchase_id,item_id,quantity,purchase_date,customer_id,loyalty_program,shop_id
0,538280,21873,11,2016-12-10 12:50:00,18427.0,0,Shop 3
1,538862,22195,0,2016-12-14 14:11:00,22389.0,1,Shop 2
2,538855,21239,7,2016-12-14 13:50:00,22182.0,1,Shop 3
3,543543,22271,0,2017-02-09 15:33:00,23522.0,1,Shop 28
4,543812,79321,0,2017-02-13 14:40:00,23151.0,1,Shop 28


In [43]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105335 entries, 0 to 105334
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   purchase_id      105335 non-null  object 
 1   item_id          105335 non-null  object 
 2   quantity         105335 non-null  int64  
 3   purchase_date    105335 non-null  object 
 4   customer_id      69125 non-null   float64
 5   loyalty_program  105335 non-null  int64  
 6   shop_id          105335 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 5.6+ MB


In [35]:
#looking at missing values
retail.isnull().sum()

purchase_id            0
item_id                0
quantity               0
purchase_date          0
customer_id        36210
loyalty_program        0
shop_id                0
dtype: int64

In [ ]:
#looking at percentage of missing values
retail['customer_id'].isnull().sum()/len(retail)

34% of the data is missing in customer id column. 

In [40]:
retail.duplicated().sum()

1033

In [42]:
retail.describe()

,quantity,customer_id,loyalty_program
count,105335.000000,69125.000000,105335.000000
mean,7.821218,21019.302047,0.226345
std,327.946695,1765.444679,0.418467
min,-74216.000000,18025.000000,0.000000
25%,0.000000,19544.000000,0.000000
50%,2.000000,20990.000000,0.000000
75%,7.000000,22659.000000,0.000000
max,74214.000000,23962.000000,1.000000


We can further the data preprocessing by taking some steps:
- filling in the missing values or drop missing values 
- take a closer look at duplicate values 
- convert data types in the retail dataset
- consider if these two tables can be joined 
- consider making a new column for total sales per customer 

### EDA
Here we can:
- take a closer look at the dates - which days of the week have the most sales & which month has the most sales
- which shop has the most sales and for which day/ month 
- which shop has the most sales for loyalty and non-loyalty members 
- look at loyalty and non-loyalty member sales over time (do loyalty members shop more?) 
- is there a certain product that is most purchased? is it by a loyalty member? 
- also providing graphs for all the above as a visual aid would help with deicision making 
- perform cohort analysis of loyalty members 
- Look into the relationship between revenue growth and the probability of churn.
- Identify the relationship between payment frequency and the probability of churn.
- Compare the time since the last purchase with the probability of churn.
- For each customer, find the date of the last purchase.
- Use this data to split the customers into n categories.
- For each category, calculate the share of the customers who left.
- Within each category, define extra indices (e.g. total sum of payments, total number of purchases).
- Draw conclusions: how time since the last purchase relates to customers' indices.
- Draw conclusions: how time since the last purchase relates to churn.

### Statistical Data Analysis 
Here we would perform hypothesis testing such as :
1. A customer is less likely to churn if they are a loyalty member. 
2. Loyalty members are more likely to spend more money. 

### Conclusion
Conclusion section will be here based on all the findings. We will then determine if being a loyalty program helped increased revenue. We will also make recommendations based on the findings. 